# Makine Öğrenimi Modelleme Süreci
1. Veri Kümesini indir. -> **Data Collection**
2. Gerekli / Gereksiz bağımsız değişkenleri bul. -> **Feature Selection**
3. 5 Farklı Yönteme göre regresyon modellerini çıkar. -> **Model Training**
    - MLR, PR, SVR, DT, RF
4. Yöntemlerin başarılarını karşılaştır. -> **Model Evaluation**
5. 10 yıl tecrübeli ve 100 puan almış bir CEO ve aynı özelliklere sahip bir Müdürün maaşlarını 5 yöntemle de tahmin edip sonuçları yorumla. -> **Prediction / Inference** + **Interpretation**

## Adjusted R-Square
Adjusted R-Square şu iki şeye bakar:
1. R² değeri: Model veriyi ne kadar açıklıyor?
2. Kullanılan değişken sayısı: Kaç tane bağımsız değişken kullandın?

- Eğer yeni değişken gerçekten modelin hatasını azaltıyorsa, Adjusted R² artar.
- Eklenen değişken, modelin açıklayıcılığını arttırmıyor ve yalnızca modelin kompleksitesini gereksiz yere yükseltiyorsa, Adjusted R² azalır.

# P-Value (probability value)
- P-değeri, elde edilen sonuçların tesadüfi mi yoksa istatistiksel olarak anlamlı mı olduğunu ölçer.
- Yorum: 
    - __p-değeri < 0.05:__ Sonuç anlamlıdır; değişken ile hedef değişken arasında ilişki vardır.
    - __p-değeri > 0.05:__ Sonuç anlamlı değildir; değişken model için önemli olmayabilir.
**P-değeri küçükse, değişkenin modele katkı sağladığı kabul edilir.**
## P-Value Teknik Açıklama
__Şu iki hipotez arasında karar vermeye yardım eder:__
- H₀ (null hypothesis): İlişki yok. (Değişkenin etkisi yok)
- H₁ (alternatif hypothesis): İlişki var. (Değişkenin etkisi var)
__P-Value çok küçükse:__
- H₀ reddedilir.
- "Evet bu değişken gerçekten önemli"
## Uygulamada P-Value Nerede Kullanılır?
- Regresyon analizinde: Bağımsız değişkenin hedefi etkileyip etkilemediğini görmek için
- A/B testlerinde: İki grup arasında anlamlı fark var mı diye kontrol etmek için
- Geriye eleme gibi model sadeleştirme yöntemlerinde


# GERİYE ELEME (Backward elimination)
**Geriye eleme, modeli sadeleştirmek için kullanılan bir özellik seçimi (feature selection) yöntemidir.**
__ADIMLAR__
1. Önem düzeyi (Significance Level, SL) seçilir (genellikle 0.05 alınır).
2. Tüm bağımsız değişkenlerle model oluşturulur.
3. En yüksek p-değerine sahip değişken belirlenir.
4. Eğer bu p-değeri p > SL ise, ilgili değişken modelden çıkarılır.
5. Model yeniden eğitilir ve 3. adıma dönülür.
6. Tüm değişkenlerin p-değerleri SL'den küçük olduğunda süreç tamamlanır.
- **Bu yöntem sayesinde, yalnızca istatistiksel olarak anlamlı değişkenler modele dahil edilir.**

|Özellik |R-Square |Adjusted R-Square |
|:-----|:----:|----:|
|Tanım |Modelin açıklama gücünü ölçer |Modelin açıklama gücünü değişken sayısını dikkate alarak ölçer |
|Değişken Eklenince Ne Olur? |Her zaman artar veya aynı kalır |Faydalıysa artar, faydasızsa düşer |
|Ne Zaman Kullanılır? |Basit modellerde (az değişkenli) |Çok değişkenli modellerde (özellikle 3+ değişken varsa) |
|Amaç |Veriye ne kadar iyi uyduğunu gösterir |Veriye iyi uyumu ve modelin aşırı şişmesini birlikte değerlendirir |

In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.metrics import r2_score

veriler = pd.read_csv("maaslar_yeni_veriler.csv")
veriler



,Calisan_ID,Unvan,UnvanSeviyesi,Kidem,Puan,Maas
0,1,Cayci,1,5,70,2250
1,2,Sekreter,2,5,70,2500
2,3,Uzman Yardimcisi,3,5,70,3000
3,4,Uzman,4,5,70,4000
4,5,Proje Yoneticisi,5,5,70,5500
5,6,Sef,6,5,70,7500
6,7,Mudur,7,5,70,10000
7,8,Direktor,8,5,70,15000
8,9,C-level,9,5,70,25000
9,10,CEO,10,5,70,50000


In [45]:
# Unvan Seviyesi, Kıdem ve Puan -> Bağımsız Değişkenler
# Unvan Dummy variable, çünkü unvan seviyesi zaten var.
# Eğer Unvan Seviyesi olmasaydı Unvan Label-Encoding yapılabilirdi.
x = veriler.iloc[:,2:5]  # R2: 0.903 - Adj. R-squared: 0.892
#x = veriler.iloc[:,2:3] # R2: 0.942 - Adj. R-squared: 0.940
x

,UnvanSeviyesi,Kidem,Puan
0,1,5,70
1,2,5,70
2,3,5,70
3,4,5,70
4,5,5,70
5,6,5,70
6,7,5,70
7,8,5,70
8,9,5,70
9,10,5,70


In [46]:
# Bağımlı Değişken : y
y = veriler.iloc[:,5:]
y

,Maas
0,2250
1,2500
2,3000
3,4000
4,5500
5,7500
6,10000
7,15000
8,25000
9,50000


In [53]:
X = x.values
Y = y.values
# Korelasyon, sayısal değişkenlerin birlikte ne kadar değiştiğini ölçer
# Örneğin: "Deneyim artınca maaş da artıyor mu?", "Puan yüksekse maaş yüksek mi?"
# 0.7 üzeri korelasyonlar genelde güçlü kabul edilir.
# 0.1 - 0.3 arası olanlar zayıf korelasyonlardır, dikkatle yorumlanmalıdır.
print(veriler[['Calisan_ID','UnvanSeviyesi','Kidem','Puan','Maas']].corr())

               Calisan_ID  UnvanSeviyesi     Kidem      Puan      Maas
Calisan_ID       1.000000       0.331847  0.206278 -0.251278  0.226287
UnvanSeviyesi    0.331847       1.000000 -0.125200  0.034948  0.727036
Kidem            0.206278      -0.125200  1.000000  0.322796  0.117964
Puan            -0.251278       0.034948  0.322796  1.000000  0.201474
Maas             0.226287       0.727036  0.117964  0.201474  1.000000


## Korelasyon Sonucu:
- UnvanSeviyesi maaşın en güçlü belirleyicisi.
- Puan ve Kıdem çok zayıf pozitif etkiler taşıyor.
- Calisan_ID gibi kimlik sütunları modelden çıkarılmalı çünkü anlamsız gürültü yaratabilir.

### Not:
- Korelasyon neden-sonuç demek değildir. Yani UnvanSeviyesi ve Maaş birlikte artmış diyebiliriz.
- __Korelasyon katsayısı iki değişkenin birlikte nasıl değiştiğini gösterir.__

|Korelasyon Katsayısı (r) |Anlamı |
|:-----|:----:|
|+1 |Mükemmel pozitif ilişki (biri artarsa diğeri de artar) |
|0 |Hiç ilişki yok |
|-1 |Mükemmel negatif ilişki (biri artarsa diğeri azalır) |

In [33]:
# Linear Regression
from sklearn.linear_model import LinearRegression # Scikit-learn kütüphanesinin içindeki LinearRegression class'ı dahil edildi.
lin_reg = LinearRegression() # LinearRegression sınıfından bir nesne oluşturuldu. Bu nesne şuan boş bir model. Henüz veri ile eğitilmedi.
lin_reg.fit(X,Y) # Verilerle eğitildi.
#OLS
#import statsmodels.api as sm
print("Linear Regression")
model = sm.OLS(lin_reg.predict(X),X)
print(model.fit().summary())
#R2
lr_r2 = r2_score(Y, lin_reg.predict(X))

Linear Regression
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.903
Model:                            OLS   Adj. R-squared (uncentered):              0.892
Method:                 Least Squares   F-statistic:                              83.89
Date:                Tue, 29 Apr 2025   Prob (F-statistic):                    8.38e-14
Time:                        19:46:31   Log-Likelihood:                         -295.74
No. Observations:                  30   AIC:                                      597.5
Df Residuals:                      27   BIC:                                      601.7
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------

In [36]:
# Polynomial Regression
# 4. Polynomial Regression (degree=2) 
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree = 2) # 2. Dereceden Polinom Eğri
x_poly = poly_reg.fit_transform(X) # X verisini polinom hale çevir.
#print(x_poly)
lin_reg2 = LinearRegression()
lin_reg2.fit(x_poly,y) # Polinom modele fit et.
print("Polynomial Degree: 2")
#OLS
model_poly2 = sm.OLS(lin_reg2.predict(poly_reg.fit_transform(X)),X)
print(model_poly2.fit().summary())

Polynomial Degree: 2
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.729
Model:                            OLS   Adj. R-squared (uncentered):              0.698
Method:                 Least Squares   F-statistic:                              24.16
Date:                Tue, 29 Apr 2025   Prob (F-statistic):                    8.31e-08
Time:                        19:49:36   Log-Likelihood:                         -314.41
No. Observations:                  30   AIC:                                      634.8
Df Residuals:                      27   BIC:                                      639.0
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------

In [37]:
# 5. Polynomial Regression (degree = 4)
from sklearn.preprocessing import PolynomialFeatures # 4. Dereceden polinom
poly_reg = PolynomialFeatures(degree = 4)
x_poly = poly_reg.fit_transform(X)
#print(x_poly)
lin_reg2 = LinearRegression()
lin_reg2.fit(x_poly,y)
print("Polynomial Degree 4")
model_poly4 = sm.OLS(lin_reg2.predict(poly_reg.fit_transform(X)),X)
print(model_poly4.fit().summary())

Polynomial Degree 4
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.680
Model:                            OLS   Adj. R-squared (uncentered):              0.644
Method:                 Least Squares   F-statistic:                              19.11
Date:                Tue, 29 Apr 2025   Prob (F-statistic):                    7.49e-07
Time:                        19:50:08   Log-Likelihood:                         -317.93
No. Observations:                  30   AIC:                                      641.9
Df Residuals:                      27   BIC:                                      646.1
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------

In [38]:
# 6. SVR - Support Vector Regression
# Not: SVR mesafe bazlı çalıştığı için verilerin ölçeklenmesi gerekir. Veriler aynı ölçekte olmazsa anlamlı tahminler yapamaz.
from sklearn.preprocessing import StandardScaler
sc1=StandardScaler()
x_olcekli = sc1.fit_transform(X) # X verisini scale et
sc2=StandardScaler()
y_olcekli = np.ravel(sc2.fit_transform(Y.reshape(-1,1))) # Y verisini scale et
# SVR Modeli Kur
from sklearn.svm import SVR
svr_reg = SVR(kernel='rbf') # RBF kernel = Esnek eğri
svr_reg.fit(x_olcekli,y_olcekli) # Modele fit et
#OLS
print("Support Vector")
model_svr = sm.OLS(svr_reg.predict(x_olcekli),x_olcekli)
print(model_svr.fit().summary())

Support Vector
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.782
Model:                            OLS   Adj. R-squared (uncentered):              0.758
Method:                 Least Squares   F-statistic:                              32.37
Date:                Tue, 29 Apr 2025   Prob (F-statistic):                    4.34e-09
Time:                        19:50:33   Log-Likelihood:                        -0.92453
No. Observations:                  30   AIC:                                      7.849
Df Residuals:                      27   BIC:                                      12.05
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------

In [40]:
from sklearn.tree import DecisionTreeRegressor
r_dt = DecisionTreeRegressor(random_state=0)
r_dt.fit(X,Y)
#OLS
print("Decision Tree")
model_dt = sm.OLS(r_dt.predict(X),X)
print(model_dt.fit().summary())

Decision Tree
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.679
Model:                            OLS   Adj. R-squared (uncentered):              0.644
Method:                 Least Squares   F-statistic:                              19.08
Date:                Tue, 29 Apr 2025   Prob (F-statistic):                    7.62e-07
Time:                        19:51:54   Log-Likelihood:                         -317.95
No. Observations:                  30   AIC:                                      641.9
Df Residuals:                      27   BIC:                                      646.1
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

In [41]:
from sklearn.ensemble import RandomForestRegressor
# n_estimators = çizilecek decision tree sayısı
rf_reg = RandomForestRegressor(n_estimators = 10, random_state=0)
rf_reg.fit(X,Y.ravel()) # Y.ravel(): 2D array olan Y'yi 1D'ye çeviriyor.
# Her ağaç kendi verisinden ve özelliklerinden öğrenir.
# Sonuçta ağaçların tahminleri ortalama alınarak final karar verilir.
#OLS
print("Random Forest")
model_rf = sm.OLS(rf_reg.predict(X),X) #OLS ordinary least squares yöntemi ile hesaplar
print(model_rf.fit().summary())

Random Forest
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.713
Model:                            OLS   Adj. R-squared (uncentered):              0.681
Method:                 Least Squares   F-statistic:                              22.31
Date:                Tue, 29 Apr 2025   Prob (F-statistic):                    1.79e-07
Time:                        19:52:14   Log-Likelihood:                         -316.07
No. Observations:                  30   AIC:                                      638.1
Df Residuals:                      27   BIC:                                      642.3
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

# Tek Parametreli Olarak
|Model |R-Square |Adjusted R-Square |
|:-----|:----:|----:|
|Linear |0.942 |0.940 |
|Polynomial Deg:2 |0.810 |0.803 |
|Polynomial Deg:4 |0.759 |0.751 |
|SVR |0.770 |0.762 |
|Decision Tree |0.751 |0.742 |
|Random Forest |0.719 |0.709 |

# 3 Parametreli Olarak
|Model |R-Square |Adjusted R-Square |
|:-----|:----:|----:|
|Linear |0.903 |0.892 |
|Polynomial Deg:2 |0.729 |0.698 |
|Polynomial Deg:4 |0.680 |0.644 |
|SVR |0.782 |0.758 |
|Decision Tree |0.679 |0.644 |
|Random Forest |0.713 |0.681 |

# ÖZET
|Konu |Tanım |
|:-----|:----:|
|Adjusted R-Square |Modelin karmaşıklığını ve açıklayıcılığını birlikte değerlendirir |
|P-Value |Değişkenin modele anlamlı katkı yapıp yapmadığını ölçer. |
|Backward Elimination |Önemsiz değişkenleri modelden sistematik olarak çıkararak sade bir model oluşturur. |

## P-Value Araştırma
1. Olasılık -> "Bir olayın olasılığı nedir?"
2. Normal Dağılım -> "Veriler doğada nasıl dağılır?"
3. Hipotez Testi -> "İddia nasıl test edilir?"
4. Z ve t testleri -> "Örneklem üzerinden genel yargı nasıl çıkarılır?"
5. Anlamlılık Seviyesi -> "0.05 kuralı nedir ve neden var?"